# Analyzing A Movie Review Dataset(Part 2)[100 Points]

## 0-A. Set-up[-2 if author function is not called]

In [1]:
def author(gt_username = 'tpasumarthi3'):
  print("This assignment is submitted by {0}.".format(gt_username))
#Add your GT_UserName below and uncomment the line.
author()

This assignment is submitted by tpasumarthi3.


In [2]:
import nltk.data
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
#nltk.download('punkt')

In [3]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

## 0-B. Text Preprocessing[10(5+5) Points]

In [4]:
# Read the train,test and unlabeled data from files and store them into variables train,test and unlabeled_train respectively.
### Add your code here. 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
#print ("Read %d labeled train reviews, %d labeled test reviews, " \
# "and %d unlabeled reviews\n" % (train["review"].size,  
# test["review"].size, unlabeled_train["review"].size ))
######################

In [5]:
def clean_review(review, remove_stopwords = False):
    """Helper function to clean the reviews i.e. to convert a document to a sequence of words.
     Please note that we're not removing stopwords since word2vec relies on the broader context
     of the sentence in order to produce high-quality word vectors.

     Arg: review: review string (str)
          remove_stopwards: If true remove stopwords else not. (boolean)
     Returns: cleaned_review : Cleaned review (list)

     You should carry out the following steps.
     1. Remove HTML Tags.
     2. Remove non-letter characters.
     3. Convert to lower case.
    """
    ### Add your code here.
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    cleaned_review= words

    #####################
    
    return cleaned_review

In [6]:
def review_to_sentences( review: str, tokenizer: nltk.tokenize.punkt.PunktSentenceTokenizer ):
    """Helper function to split a review into parsed sentences. Returns a 
     list of sentences, where each sentence is a list of words.

     Arg: review: review string (str)
          tokenizer: punkt tokenizer
     Returns:
          review_sentences: List of list of tokens.
                            e.g. [["word2vec", "was", "introduced", "by", "google" ],["it","leverages","distributed","token","representations"]]

     You should carry out the following steps.
     1. Use the tokenizer to split the paragraph into sentences.
     2. Clean the sentence to return a list of words for each sentence using the helper funtion above.
     3. Return a list of tokenized sentences.
    """
    ### Add your code here.
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(clean_review(raw_sentence))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    review_sentences= sentences
    ######################
    
    return review_sentences

In [7]:
# Use the above helper functions to convert the reviews in train["review"] and in unlabeled_train["review"] to a list of list format as mentioned above.
# For example if your train data contains 2 reviews with 3 sentences in each and the unlabeled_train has 4 reviews with 1 sentence in each
# The resultant list should have 10 lists of tokenized sentences.
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
### Add your code here.
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)
######################

print("Parsing sentences from unlabeled set")
### Add your code here.
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)
######################

Parsing sentences from training set


C:\Users\tyco9\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\tyco9\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\tyco9\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


C:\Users\tyco9\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\tyco9\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\tyco9\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'... ...'" looks like a filename, not markup. You should probably open this file and pass the filehand

## 1. Word2Vec[80(20*4) Points]

### a. Create vector representations for each movie post in your training set by training word2vec with context=5, embedding dimension=100, min_words=40. We’ll call the collection of these representations Z1.[20 Points]

In [8]:
from gensim.models import word2vec, KeyedVectors

In [9]:
def generate_z1(sentences,num_features,min_word_count,context,num_workers = 4,downsampling = 1e-3, model_name = "model1_100features_40minwords_5context"):
    """Set values for the parameters of the your word2vec model and train it on the extracted sentences from both the train and unlabeled_train data and
      generate the collection of these representations Z1.
      You should carry out the following.
      1) Set the parameters as mentioned below. 
        A)Constrained Paramters : "context length", "embedding dimension", "min_words" (Please check the question for the values.)
        B)Optional Parameters: "number of workers", "downsample setting"
      2) Train your word2vec model and save it.
      3) Store the collection of word embeddings and the word_list(z1 and word_list_z1) .

      Arg: sentences: List of tokenized sentences (List)
            num_features: Word vector dimensionality (int)
            min_word_count: Minimum word count (int)
            context: Context window size (int)
            num_workers: Number of threads to run in parallel (int)
            downsampling: Downsample setting for frequent words (float)
            model_name = Name to save your model (str)
      Returns:
            trained_word2vec_model: word2vec model trained on the tokenized sentences.
            z1: word embeddings (ndarray)
            word_list_z1: List of tokens in the model (List)

      

    """
    print("Training model...")
    ### Add your code here.
    
    # Initialize and train the model (this will take some time)
    trained_word2vec_model = word2vec.Word2Vec(sentences, workers=num_workers, \
                size=num_features, min_count = min_word_count, \
                window = context, sample = downsampling)

    # If you don't plan to train the model any further, calling 
    # init_sims will make the model much more memory-efficient.
    trained_word2vec_model.init_sims(replace=True)

    # It can be helpful to create a meaningful model name and 
    # save the model for later use. You can load it later using Word2Vec.load()
    model_name = "100features_40minwords_5context"
    trained_word2vec_model.save(model_name)
    
    z1 = trained_word2vec_model.wv.syn0
    word_list_z1 = trained_word2vec_model.wv.index2word
    
    #######################

    return trained_word2vec_model, z1, word_list_z1

# Set values for the parameters of the your word2vec model and train it on the extracted sentences from both the train and unlabeled_train data and
# generate the collection of these representations Z1.
# Add the function parameter values below.
### Add your code here.

# Set values for various parameters
num_features = 100    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                 
context = 5          # Context window size                   

######################

model1, z1, word_list_z1 = generate_z1(sentences,num_features,min_word_count,context)




Training model...


C:\Users\tyco9\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).


### b. Create vector representations for each movie post in your training set by loading the pretrained Google word2vec model. We’ll call the collection of these representations Z2.[20 Points]

In [10]:
def generate_z2(word_list_z1):
    """Use a pre-trained word2vec model(GoogleNews-vectors-negative300) to generate the collection of these representations Z2. Please note Z2 corresponds to the word embeddings for
      words that are present in both the previous model and this pre-trained model.

      You should carry out the following.

      Arg: word_list_z1: List of tokens in the previous trained model (List)
      Returns:
            pre_trained_word2vec_model: word2vec model trained on the tokenized sentences.
            z2: word embeddings (ndarray)
            word_list_z2: List of tokens in the model (List)   

    """
    ### Add your code here.
    pre_trained_word2vec_model= KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
    p_list= pre_trained_word2vec_model.index2word
    word_list_z2= [word for word in word_list_z1 if word in p_list]
    z2= np.zeros((len(word_list_z2), 300))
    for i, word in enumerate(word_list_z2):
        z2[i,:]= pre_trained_word2vec_model[word]
    ######################
    return pre_trained_word2vec_model, z2, word_list_z2

model2, z2, word_list_z2 = generate_z2(word_list_z1)

### c. With k=10, do k-means clustering on each set Z1, Z2. Print a table of the words in each cluster for Z1 and for Z2.[20 Points]

In [11]:
from sklearn.cluster import KMeans

In [12]:
def fit_kmeans(z,word_list_z,num_clusters = 10):
    """ Fit kmeans on the embedding representations and return a mapping of word to cluster indices. Please use the default values for
        the rest of the kmeans parameters.

        Arg: z: word embeddings (ndarray)
              word_list_z: List of tokens in the model (List) 
              num_clusters: Number of clusters (int)
        Returns:
            pre_trained_word2vec_model: word2vec model trained on the tokenized sentences.
            z2: word embeddings (ndarray)
            word_centroid_map_z: A mapping of word to cluster index it belongs to. (Dict)      

    """
    ### Add your code here.
    kmeans_clustering= KMeans(n_clusters= num_clusters)
    pred= kmeans_clustering.fit_predict(z)
    word_centroid_map_z = dict(zip(word_list_z, pred))
    ######################

    return word_centroid_map_z


word_centroid_map_z1 = fit_kmeans(z1, word_list_z1, 10)
word_centroid_map_z2 = fit_kmeans(z2, word_list_z2, 10)

In [13]:
def print_clusters(word_centroid_map_z, model_name):
    """ Print max(20, cluster_size) words for each of the clusters.

        Args: word_centroid_map_z: A mapping of word to cluster index it belongs to. (Dict)  
              model_name: Model Name (str)


    """
    print("The clusters for {0} are....".format(model_name))
    ### Add your code here.
    k= len(list(set(word_centroid_map_z.values())))
    for c in range(k):
        i= 0
        words= []
        for key, value in word_centroid_map_z1.items():
            if(i > 20):
                break
            if(value == c):
                words.append(key)
                i=i+1
        print(words)
        print()
    print()
    ######################
print_clusters(word_centroid_map_z1, "model1")
print_clusters(word_centroid_map_z2, "model2")


The clusters for model1 are....
['by', 'american', 'star', 'black', 'classic', 'early', 'directed', 'late', 'writer', 'including', 'novel', 'modern', 'british', 'king', 'french', 'york', 'general', 'e', 'among', 'th', 'de']

['death', 'house', 'dead', 'face', 'head', 'white', 'run', 'hand', 'etc', 'car', 'fight', 'blood', 'running', 'room', 'body', 'red', 'killing', 'dog', 'monster', 'hot', 'gun']

['john', 'mr', 'michael', 'david', 'robert', 'james', 'o', 'jack', 'george', 'richard', 'tom', 'lee', 'paul', 'peter', 'dr', 'joe', 'bill', 'mark', 'william', 'harry', 'co']

['their', 'our', 'war', 'men', 'women', 'against', 'town', 'city', 'group', 'girls', 'police', 'living', 'order', 'middle', 'country', 'local', 'team', 'america', 'space', 'eventually', 'crew']

['his', 'who', 'her', 'man', 'young', 'girl', 'woman', 'wife', 'himself', 'father', 'mother', 'boy', 'friend', 'son', 'evil', 'killer', 'child', 'daughter', 'hero', 'husband', 'murder']

['the', 'and', 'a', 'to', 'is', 'it', 'in

### d. Featurize the training and test reviews in Z1, Z2 to produce design matrices X1,X2 as described in part 3 of the blog series. Basically, each review is converted into a bag of centroids feature vector with each vector component representing the count of the number of words in that review that belong in that component’s cluster.[20 Points]

In [14]:
# Create tokenized reviews for both train and test. Please note that for this we'll not be splitting the sentences and we'll be removing stopwords.
# This is exactly what you did in the preprocessing step for HW1
clean_train_reviews = []
clean_test_reviews = []
### Add your code here.
for review in train["review"]:
    clean_train_reviews.append(clean_review(review, remove_stopwords=True))
    
for review in test["review"]:
    clean_test_reviews.append(clean_review(review, remove_stopwords=True))
#######################

In [15]:
def create_bag_of_centroids(tokenized_review, word_centroid_map, num_clusters = 10):
    """ Create a bag of kmeans centroids for each review i.e. for a review we return an array of length num_clusters with each 
        element of the array indicating how many words(tokens) of the review belong to that cluster.

        Args: tokenized_review: list of tokens corresponding to a review (List)
              word_centroid_map: word to cluster_index map for the model (Dict)
              num_clusters: Number of clusters (int)
        
        Returns: bag_of_centroids: An array containing the count of tokens in each cluster (ndarray)
    """
  
    ### Add your code here.
    bag_of_centroids= np.zeros(num_clusters)
    
    for word in tokenized_review:
        if word in word_centroid_map:
            idx= word_centroid_map[word]
            bag_of_centroids[idx]= bag_of_centroids[idx]+1
    #######################
    return bag_of_centroids

In [23]:
def create_design_matrices(data, cleaned_reviews, word_centroid_map_z1, word_centroid_map_z2, num_clusters = 10):
    """ Creates the design matrices X1(for trained model) and X2(for pretrained google word2vec model) for the given data

        Args: data: Train/Test data (pandas.core.frame.DataFrame)
              cleaned_reviews: List of tokenized reviews(sentences are not split and stopwords removed) (List)
              word_centroid_map_z1: word to cluster map for model1 (Dict)
              word_centroid_map_z2: word to cluster map for model2 (Dict)
              num_clusters: Number of KMeans clusters (int)

        Returns:
              x1_data: Design matrices X1-- Shape should be num_reviews*num_clusters (np.ndarray) 
              x2_data: Design matrices X2-- Shape should be num_reviews*num_clusters (np.ndarray)


    """
    ### Add your code here.
    x1_data= np.zeros((data["review"].size, num_clusters))
    x2_data= np.zeros((data["review"].size, num_clusters))
    
    for i, review in enumerate(cleaned_reviews):
        x1_data[i]= create_bag_of_centroids(review, word_centroid_map_z1)
        x2_data[i]= create_bag_of_centroids(review, word_centroid_map_z2)
    ######################
    return x1_data,x2_data
x1_train,x2_train = create_design_matrices(train,clean_train_reviews,word_centroid_map_z1,word_centroid_map_z2,10)
x1_test,x2_test = create_design_matrices(test,clean_test_reviews,word_centroid_map_z1,word_centroid_map_z2,10)



### e. Save X1, X2 for both train and test set.[-10 if missing]

In [24]:
#Save the design matrices for both train and test set as .npy files and submit these as part of the delieverables
### Add your code here.
np.save('x1_train.npy', x1_train)
np.save('x1_test.npy', x1_test)
np.save('x2_train.npy', x2_train)
np.save('x2_test.npy', x2_test)
######################

## 2. Classification Experiment[10(5+3+2) Points]

### a. Properly train and tune a collection of random forest classifiers using cross-validation for the design matrices X1 and X2. You should end up with two classifiers, M1 and M2. Print the best f1_scores on the test set for both the design matrices.[8 Points]

In [25]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [26]:
#Get the value for y from the original dataframe
y_train = train['sentiment'].as_matrix()

C:\Users\tyco9\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [27]:
def split(X, label):
    """Helper function to create a train-test split of the data.(test_size = 0.2, random_state = 0)

     Arg: X: Design Matrix(ndarray)
          label: sentiment values
     Returns:
          x_train: train input features
          x_test: test input features
          y_train: train labels
          y_test: test labels
    """
    ###Add your code here.
    x_train, x_test, y_train, y_test= train_test_split(X, label, test_size=0.2, random_state=0)
    #####################
    return x_train, x_test, y_train, y_test

In [28]:
# Call split function for both the design matrices x1 and x2 obtained from Q1. 
###Add your code here.
x1_train_split, x1_test_split, y1_train_split, y1_test_split= split(x1_train, y_train)
x2_train_split, x2_test_split, y2_train_split, y2_test_split= split(x2_train, y_train)
#####################

In [29]:
def cross_val(x_train_split,y_train_split,n_estimators):
    """function to calculate the cross validation scores for the 2 values of x_train, and y_train on the Random 
        Forest Classifier.

     Arg: x_train_split: train input features
          y_train_split: train labels
          n_estimators: List of estimators to perform Random Forest with.
     Returns:
          cross_val_scores: List of the CV scores the estimators (list)
    """
    
    ###Add your code here.
    cross_val_scores= []
    for n in n_estimators:
        clf= RandomForestClassifier(n_estimators= n)
        scores= cross_val_score(clf, x_train_split, y_train_split)
        cross_val_scores.append(scores.mean())
    #####################
    return cross_val_scores
n_estimators = [50, 100, 150, 200, 300, 400, 500, 750, 1000]
result1 = cross_val(x1_train_split,y1_train_split,n_estimators) #CV scores for X1
result2 = cross_val(x2_train_split,y2_train_split,n_estimators) #CV scores for X2

C:\Users\tyco9\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\tyco9\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\tyco9\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\tyco9\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split

In [30]:
def cal_f1(x_train_split,y_train_split,x_test_split,y_test_split,result,n_estimators):
    """function to calculate the best F-1 score based on the best value of n_estimators obtained from cross-val.

     Arg: x_train_split: train input features
          y_train_split: train labels
          x_test_split: test input features
          y_test_split: test labels
          n_estimators: List of estimators to perform Random Forest with.
     Returns:
          f1: f1_score for the best value of n_estimator obtained from cross-val (float)
    """
    ###Add your code here.
    n= n_estimators[result.index(max(result))]
    clf= RandomForestClassifier(n_estimators= n)
    clf.fit(x_train_split, y_train_split)
    y_pred= clf.predict(x_test_split)
    f1= f1_score(y_test_split, y_pred)
    #####################
    return f1
f1_first = cal_f1(x1_train_split,y1_train_split,x1_test_split,y1_test_split,result1,n_estimators)
f1_second = cal_f1(x2_train_split,y2_train_split,x2_test_split,y2_test_split,result2,n_estimators)
print("The best f1 score using X1 as input is {0}".format(f1_first))
print("The best f1 score using X2 as input is {0}".format(f1_second))


The best f1 score using X1 as input is 0.6400961345884238
The best f1 score using X2 as input is 0.7718298392021169


### b. Which featurization technique works best for sentiment classification? Is this better or worse than the simple bag-of-words approach? What are at least three things you could do to improve the efficacy of the classifier?[2 Points]

The pretrained google word2vec model works better than our trained model, as it has a higher F1 score. This is much better than the bag-of-words model. Three thing that we could do to improve the efficacy is increasing the amount of data we used to train the model, incorporating ensebles learning methods with boosting, and increasing the embedding dimensions.